# Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/ReverseEmbedding/GEIA

Mounted at /content/drive/
/content/drive/MyDrive/ReverseEmbedding/GEIA


In [3]:
from tqdm import tqdm

In [4]:
!pip install bert_score rouge_score xlsxwriter

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 16.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8bfd68b0f264bc9fcc32c3c256e87ed0f47948321c2c7e505c910103942bb56a
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [5]:
#@title Load libraries and initialize the model (attacker_model), embedding model (model), projection, tokenizer and sent_list
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# import the libraries
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.spatial.distance import cosine

import json
import numpy as np
import pandas as pd
import argparse
import sys
import random

from torch.optim import AdamW
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from attacker_models import SequenceCrossEntropyLoss
from sentence_transformers import SentenceTransformer
from simcse_persona import get_persona_dict
from attacker_evaluation_gpt import eval_on_batch
from datasets import load_dataset
from data_process import get_sent_list

class linear_projection(nn.Module):
    def __init__(self, in_num, out_num=1024):
        """Mapping between embedding dimension and GPT token representation dimension"""
        super(linear_projection, self).__init__()
        self.fc1 = nn.Linear(in_num, out_num)

    def forward(self, x, use_final_hidden_only = True):
        # x should be of shape (?,in_num) according to gpt2 output
        out_shape = x.size()[-1]
        assert(x.size()[1] == out_shape)
        out = self.fc1(x)


        return out


class personachat(Dataset):
    def __init__(self, data):
        self.data = data


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        text = self.data[index]

        return  text

    def collate(self, unpacked_data):
        return unpacked_data

def process_data(data,batch_size,device,config,need_porj=True):
    #model = SentenceTransformer('all-roberta-large-v1',device=device)   # dim 1024
    device_1 = torch.device("cuda:0")
    model = SentenceTransformer(config['embed_model_path'],device=device_1)   # dim 768
    dataset = personachat(data)
    dataloader = DataLoader(dataset=dataset,
                              shuffle=True,
                              batch_size=batch_size,
                              collate_fn=dataset.collate)

    print('load data done')
    ### extra projection
    if need_porj:
        """This is automatically true"""
        projection = linear_projection(in_num=768, out_num=1280).to(device)
        if config.get('resume', False):
          proj_path = f"models/projection_diablogpt_large_{config['dataset']}_{config['embed_model']}"
          if os.path.exists(proj_path):
              projection.load_state_dict(torch.load(proj_path))
              print(f"✅ Loaded projection from {proj_path}")
    ### for attackers

    """The default attacker model is DialoGPT-large"""
    model_attacker = AutoModelForCausalLM.from_pretrained(config['model_dir'])
    if config.get('resume', False):
      attacker_path = f"models/attacker_diablogpt_large_{config['dataset']}_{config['embed_model']}"
      if os.path.exists(attacker_path):
          model_attacker = AutoModelForCausalLM.from_pretrained(attacker_path).to(device)
          print(f"✅ Loaded attacker model from {attacker_path}")
    tokenizer_attacker = AutoTokenizer.from_pretrained(config['model_dir'])
    criterion = SequenceCrossEntropyLoss()
    model_attacker.to(device)
    param_optimizer = list(model_attacker.named_parameters())

    # print("Param optimizer length:", param_optimizer)
    no_decay = ['bias', 'ln', 'LayerNorm.weight']

    """Exclude bias and layernorm from weight decay"""
    optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}

    ]

    # print("Optimizer grouped parameters length:", optimizer_grouped_parameters)

    num_gradients_accumulation = 1
    """How many batches to accumulate before performing an optimizer step"""

    num_epochs = config['num_epochs']
    batch_size = config['batch_size']

    num_train_optimization_steps  = len(dataloader) * num_epochs // num_gradients_accumulation
    """Each batch count as one step, the total number of steps is num_epochs * number of batches"""

    optimizer = AdamW(optimizer_grouped_parameters,
                  lr=3e-5,
                  eps=1e-06)
    if need_porj:
        optimizer.add_param_group({'params': projection.parameters()})

    """This is a linear scheduler with warmup
    In the total number of training steps, the first 100 steps are used for warmup
    increasing the learning rate linearly from 0 to the initial lr set in the optimizer

    After the warmup period, the learning rate decreases linearly from the initial lr to 0"""
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=100,
                                            num_training_steps=num_train_optimization_steps)


    start_epoch = 0

    if config.get('resume', False) and os.path.exists("models/checkpoint_epoch.pt"):
        checkpoint = torch.load("models/checkpoint_epoch.pt")
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        scheduler.load_state_dict(checkpoint['scheduler_state'])
        start_epoch = checkpoint['epoch'] + 1
        print("✅ Loaded optimizer and scheduler state")

    ### process to obtain the embeddings
    print("Starting from epoch", start_epoch)

    for i in range(start_epoch, num_epochs):
        epoch_bar = tqdm(enumerate(dataloader),
                      total=len(dataloader),
                      desc=f"Epoch {i+1}/{num_epochs}",
                      unit="batch")
        model.eval()
        for idx,batch_text in epoch_bar:
            with torch.no_grad():
                embeddings = model.encode(batch_text,convert_to_tensor = True).to(device)
                print(f'Embedding dim: {embeddings.size()}')

            ### attacker part, needs training
            if need_porj:
               embeddings = embeddings.clone().detach()
               embeddings = projection(embeddings)

            record_loss, perplexity = train_on_batch(batch_X=embeddings,batch_D=batch_text,model=model_attacker,tokenizer=tokenizer_attacker,criterion=criterion,device=device,train=True)
            optimizer.step()
            scheduler.step()
            # make sure no grad for GPT optimizer
            optimizer.zero_grad()
            print(f'Training: epoch {i} batch {idx} with loss: {record_loss} and PPL {perplexity} with size {embeddings.size()}')
            #sys.exit(-1)

        """Saves after each epoch"""
        if need_porj:
            proj_path = 'models/' + 'projection_diablogpt_large_' + config['dataset'] + '_' + config['embed_model']
            torch.save(projection.state_dict(), proj_path)
        save_path = 'models/' + 'attacker_diablogpt_large_' + config['dataset'] + '_' + config['embed_model']
        print(save_path)
        model_attacker.save_pretrained(save_path)
        torch.save({
            'epoch': i,
            'optimizer_state': optimizer.state_dict(),
            'scheduler_state': scheduler.state_dict(),
        }, f"models/checkpoint_epoch.pt")


### used for testing only
def process_data_test(data,batch_size,device,config,need_proj=True):
    #model = SentenceTransformer('all-roberta-large-v1',device=device)   # dim 1024
    #model = SentenceTransformer(config['embed_model_path'],device=device)   #  dim 768
    device_1 = torch.device("cuda:0")
    print(device_1)

    """This loads the embedding model"""
    model = SentenceTransformer(config['embed_model_path'],device=device_1)   # dim 768
    if(config['decode'] == 'beam'):
        save_path = 'models/' + 'attacker_gpt2_large_' + config['dataset'] + '_' + config['embed_model']+'_beam'+'.log'
    else:
        save_path = 'models/' + 'attacker_gpt2_large_' + config['dataset'] + '_' + config['embed_model']+'.log'
    dataset = personachat(data)
    # no shuffle for testing data
    dataloader = DataLoader(dataset=dataset,
                              shuffle=False,
                              batch_size=batch_size,
                              collate_fn=dataset.collate)

    print('load data done')

    """Here it tries to load a projection model if there is a checkpoint"""
    if need_proj:
        proj_path = 'models/' + 'projection_gpt2_large_' + config['dataset'] + '_' + config['embed_model']
        projection = linear_projection(in_num=768, out_num=1280)
        projection.load_state_dict(torch.load(proj_path))
        projection.to(device)
        print('load projection done')
    else:
        print('no projection loaded')
    # setup on config for sentence generation   AutoModelForCausalLM
    attacker_path = 'models/' + 'attacker_gpt2_large_' + config['dataset'] + '_' + config['embed_model']
    config['model'] = AutoModelForCausalLM.from_pretrained(attacker_path).to(device)
    config['tokenizer'] = AutoTokenizer.from_pretrained('microsoft/DialoGPT-large')

    # sent_dict = {}
    # sent_dict['gt'] = []
    # sent_dict['pred'] = []
    # with torch.no_grad():
    #     for idx,batch_text in enumerate(dataloader):

    #         embeddings = model.encode(batch_text,convert_to_tensor = True).to(device)

    #         if need_proj:
    #             embeddings = projection(embeddings)

    #         sent_list, gt_list = eval_on_batch(batch_X=embeddings,batch_D=batch_text,model=config['model'],tokenizer=config['tokenizer'],device=device,config=config)
    #         print(f'testing {idx} batch done with {idx*batch_size} samples')
    #         sent_dict['pred'].extend(sent_list)
    #         sent_dict['gt'].extend(gt_list)

    #     with open(save_path, 'w') as f:
    #         json.dump(sent_dict, f,indent=4)

    # return 0


### used for testing only
def process_data_test_simcse(data,batch_size,device,config,proj_dir=None,need_proj=False):
    tokenizer = AutoTokenizer.from_pretrained(config['embed_model_path'])  # dim 1024
    model = AutoModel.from_pretrained(config['embed_model_path']).to(device)
    #save_path = 'logs/attacker_gpt2_qnli_simcse_bert_large.log'
    if(config['decode'] == 'beam'):
        print('Using beam search decoding')
        save_path = 'logs/' + 'attacker_gpt2_large_' + config['dataset'] + '_' + config['embed_model']+'_beam'+'.log'
    else:
        save_path = 'logs/' + 'attacker_gpt2_large_' + config['dataset'] + '_' + config['embed_model']+'.log'
    dataset = personachat(data)
    # no shuffle for testing data
    dataloader = DataLoader(dataset=dataset,
                              shuffle=False,
                              batch_size=batch_size,
                              collate_fn=dataset.collate)

    print('load data done')
    if need_proj:
        projection = linear_projection(in_num=768)
        projection.load_state_dict(torch.load(proj_dir))
        projection.to(device)
        print('load projection done')
    else:
        print('no projection loaded')
    # setup on config for sentence generation   AutoModelForCausalLM
    attacker_path = 'models/' + 'attacker_gpt2_large_' + config['dataset'] + '_' + config['embed_model']
    config['model'] = AutoModelForCausalLM.from_pretrained(attacker_path).to(device)
    config['tokenizer'] = AutoTokenizer.from_pretrained('microsoft/DialoGPT-large')

    sent_dict = {}
    sent_dict['gt'] = []
    sent_dict['pred'] = []
    with torch.no_grad():
        for idx,batch_text in enumerate(dataloader):
            inputs = tokenizer(batch_text, padding=True, truncation=True, return_tensors="pt").to(device)
            embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
            if need_proj:
                embeddings = projection(embeddings)
            #sent_list, gt_list = eval_on_batch(batch_X=embeddings,batch_D=batch_text,model=config['model'],tokenizer=config['tokenizer'],device=device,config=config)
            sent_list, gt_list = eval_on_batch(batch_X=embeddings,batch_D=batch_text,model=config['model'],tokenizer=config['tokenizer'],device=device,config=config)
            print(f'testing {idx} batch done with {idx*batch_size} samples')
            sent_dict['pred'].extend(sent_list)
            sent_dict['gt'].extend(gt_list)
        with open(save_path, 'w') as f:
            json.dump(sent_dict, f,indent=4)

    return 0


def train_on_batch(batch_X,batch_D,model,tokenizer,criterion,device,train=True):
    padding_token_id = tokenizer.encode(tokenizer.eos_token)[0]
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(batch_D, return_tensors='pt', padding='max_length', truncation=True, max_length=40)
    #dial_tokens = [tokenizer.encode(item) + turn_ending for item in batch_D]
    #print(inputs)
    input_ids = inputs['input_ids'].to(device) # tensors of input ids
    labels = input_ids.clone()
    #print(input_ids.size())
    # embed the input ids using GPT-2 embedding
    input_emb = model.transformer.wte(input_ids)
    # add extra dim to cat together
    batch_X = batch_X.to(device)
    batch_X_unsqueeze = torch.unsqueeze(batch_X, 1)
    inputs_embeds = torch.cat((batch_X_unsqueeze,input_emb),dim=1)   #[batch,max_length+1,emb_dim (1024)]
    past = None
    # need to move to device later
    inputs_embeds = inputs_embeds

    #logits, past = model(inputs_embeds=inputs_embeds,past = past)
    logits, past = model(inputs_embeds=inputs_embeds,past_key_values  = past,return_dict=False)
    logits = logits[:, :-1].contiguous()
    target = labels.contiguous()
    target_mask = torch.ones_like(target).float()
    loss = criterion(logits, target, target_mask, label_smoothing=0.02, reduce="batch")

    record_loss = loss.item()
    perplexity = np.exp(record_loss)
    if train:
        loss.backward()

    return record_loss, perplexity


#Load the model
def load_model(config):
    attacker_path = f"models/attacker_diablogpt_large_{config['dataset']}_{config['embed_model']}"
    if os.path.exists(attacker_path):
        model_attacker = AutoModelForCausalLM.from_pretrained(attacker_path).to(device)
        print(f"✅ Loaded attacker model from {attacker_path}")
    tokenizer_attacker = AutoTokenizer.from_pretrained(config['model_dir'])
    model_attacker.to(device)

    model_attacker.eval()

    projection = linear_projection(in_num=768, out_num=1280).to(device)
    proj_path = f"models/projection_diablogpt_large_{config['dataset']}_{config['embed_model']}"
    if os.path.exists(proj_path):
        projection.load_state_dict(torch.load(proj_path))
        projection.eval()
        print(f"✅ Loaded projection from {proj_path}")

    return model_attacker, tokenizer_attacker, projection

sbert = SentenceTransformer('all-MiniLM-L6-v2')  # small+fast; swap for larger if you want
def sbert_cosine(a, b):
    ea = sbert.encode(a, convert_to_tensor=False)
    eb = sbert.encode(b, convert_to_tensor=False)
    cos = np.dot(ea, eb) / (np.linalg.norm(ea) * np.linalg.norm(eb))
    return float(cos)


# 2) BERTScore
from bert_score import score

def bertscore_f1(hyps, refs, model_type='microsoft/deberta-xlarge-mnli'):
    P, R, F1 = score(hyps, refs, model_type=model_type, lang='en', rescale_with_baseline=True)
    return F1.numpy().tolist()  # list of floats


# 3) ROUGE-L
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
def rouge_l(a, b):
    return scorer.score(a, b)['rougeL'].fmeasure


times = 10
model_cards ={}
model_cards['sent_t5_large'] = 'sentence-t5-large'
model_cards['sent_t5_base'] = 'sentence-t5-base'
model_cards['sent_t5_xl'] = 'sentence-t5-xl'
model_cards['sent_t5_xxl'] = 'sentence-t5-xxl'
model_cards['mpnet'] = 'all-mpnet-base-v1'
model_cards['sent_roberta'] = 'all-roberta-large-v1'
model_cards['simcse_bert'] = 'princeton-nlp/sup-simcse-bert-large-uncased'
model_cards['simcse_roberta'] = 'princeton-nlp/sup-simcse-roberta-large'
parser = argparse.ArgumentParser(description='Training external NN as baselines')
parser.add_argument('--model_dir', type=str, default='microsoft/DialoGPT-large', help='Dir of your model')
parser.add_argument('--resume', type=bool, default=True, help='Resume training from checkpoint')
parser.add_argument('--resume_epoch', type=int, default=0, help='Epoch number to resume from')
parser.add_argument('--num_epochs', type=int, default=10, help='Training epoches.')
parser.add_argument('--batch_size', type=int, default=16, help='Batch_size #.')
parser.add_argument('--dataset', type=str, default='personachat', help='Name of dataset: personachat or qnli')
#parser.add_argument('--dataset', type=str, default='qnli', help='Name of dataset: personachat or qnli')
#parser.add_argument('--data_type', type=str, default='train', help='train/test')
parser.add_argument('--data_type', type=str, default='train', help='train/test')
parser.add_argument('--embed_model', type=str, default='sent_t5_base', help='Name of embedding model: mpnet/sent_roberta/simcse_bert/simcse_roberta/sent_t5')
parser.add_argument('--decode', type=str, default='beam', help='Name of decoding methods: beam/sampling')
#parser.add_argument('--embed_model', type=str, default='simcse_roberta', help='Name of embedding model: mpnet/sent_roberta/simcse_bert/simcse_roberta/sent_t5')

args, unknown = parser.parse_known_args()

config = {}
config['model_dir'] = args.model_dir
config['num_epochs'] = args.num_epochs
config['batch_size'] = args.batch_size
config['dataset'] = args.dataset
config['data_type'] = args.data_type
config['embed_model'] = args.embed_model
config['decode'] = args.decode
config['embed_model_path'] = model_cards[config['embed_model']]
config['device'] = torch.device("cuda")
config['tokenizer'] = AutoTokenizer.from_pretrained('microsoft/DialoGPT-large')
config['eos_token'] = config['tokenizer'].eos_token
config['use_opt'] = False
config['resume'] = True
# config['resume_epoch'] =  3

device = torch.device("cuda:0")
#device = torch.device("cpu")
batch_size = config['batch_size']
sent_list = get_sent_list(config)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

In [5]:
"""Using persona chat dataset"""
attacker_model, tokenizer, projection = load_model(config)

✅ Loaded attacker model from models/attacker_diablogpt_large_personachat_sent_t5_base
✅ Loaded projection from models/projection_diablogpt_large_personachat_sent_t5_base


In [6]:
device_1 = torch.device("cuda:0")
model = SentenceTransformer(config['embed_model_path'],device=device_1)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [6]:
#@title Set up the inference code


# def inference(sample_sentence, model, projection, attacker_model, use_sampling=False, top_k=0, noise=None, x2 = False, concat = False, batch=False):
#   """Not recommneded for batch inference unless in batch mode"""
#   model.eval()
#   projection.eval()
#   attacker_model.eval()


#   """Code will break if it is a list on input"""
#   if concat:
#     if sample_sentence.strip().endswith('.'):
#       sample_sentence = sample_sentence * 2
#     else:
#       sample_sentence = sample_sentence + ". " + sample_sentence + "."

#   if type(sample_sentence) is not list:
#     sample_sentence = [sample_sentence]
#   with torch.no_grad():
#       #  Encode and project initial embedding
#       print(f"Processing sample sentence: {sample_sentence}")
#       embeddings = model.encode(sample_sentence, convert_to_tensor=True).to(device)
#       if noise is not None:
#           # print("Original embeddings: ", embeddings)
#           # print("Noise: ", noise)
#           # a = embeddings.clone()
#           embeddings = embeddings - noise
#           # print("Noised embeddings: ", embeddings)
#           # if torch.all(embeddings + noise == a): print("True")

#       if x2:
#         embeddings = embeddings * 2
#       # print("Before shape ", embeddings.shape)
#       embeddings = projection(embeddings).unsqueeze(1)  # [batch=1, seq_len=1, hidden_dim] This is because the attacker requires a sequence dimension
#       # print("After shape ", embeddings.shape)

#       #  Initialize generation loop
#       generated_embeds = embeddings
#       past_key_values = None
#       max_new_tokens = 50   # how many tokens to generate
#       temperature = 1.0     # can adjust for diversity

#       # return generated_embeds

#       #  Iteratively generate
#       for _ in range(max_new_tokens):
#           # Forward through model
#           outputs = attacker_model(
#               inputs_embeds=generated_embeds,
#               past_key_values=past_key_values,
#               use_cache=True,
#               return_dict=True
#           )

#           logits = outputs.logits       # [batch, seq_len, vocab_size]
#           past_key_values = outputs.past_key_values

#           #  Get next-token prediction (last position)
#           next_token_logits = logits[:, -1, :]

#           if use_sampling:
#               if temperature is not None and temperature > 0:
#                   next_token_logits = next_token_logits / temperature
#               # Optional top-k filtering
#               if top_k is not None and top_k > 0:
#                   top_logits, _ = torch.topk(next_token_logits, top_k)
#                   min_logits = top_logits[:, -1].unsqueeze(-1)
#                   next_token_logits = torch.where(
#                       next_token_logits < min_logits, torch.full_like(next_token_logits, -float('Inf')), next_token_logits
#                   )

#               # Sample from probability distribution
#               probs = torch.softmax(next_token_logits, dim=-1)
#               next_token_id = torch.multinomial(probs, num_samples=1)
#           else:
#               # Greedy
#               # print(next_token_logits.shape)
#               next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)
#               # print(next_token_id.shape)

#           if _ == 0:
#               output_ids = next_token_id
#           else:
#               output_ids = torch.cat((output_ids, next_token_id), dim=1)

#           # if len(next_token_id) == 1:
#           #   """Batch size one, break when reached EOS"""
#           #   if next_token_id[0] == tokenizer.eos_token_id:
#           #     print("Break point reached")
#           #     break

#           #  Convert predicted token to embedding for next step
#           next_token_emb = attacker_model.transformer.wte(next_token_id)

#           # Only feed in the *new* token embedding next time
#           generated_embeds = next_token_emb

#           # (optional) collect IDs for printing later


#       #  Decode generated tokens
#       result = []
#       for output_id in output_ids:
#         generated_text = tokenizer.decode(output_id, skip_special_tokens=True) #This only outputs the first one
#         result.append(generated_text)
#       return result

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def Differential(embedding, epsilon=10.0, sensitivity=1.0, renormalize=False):
    """
    Applies Differential Privacy to an embedding vector using the Laplace Mechanism.

    Args:
        embedding (np.array): The original embedding vector (1D or 2D array).
        epsilon (float): The privacy budget. Lower value = More privacy (more noise).
                         Higher value = Less privacy (less noise).
        sensitivity (float): The maximum amount the embedding can change.
                             Usually 1.0 or 2.0 for normalized embeddings.

    Returns:
        np.array: The noisy, private embedding.
    """
    embedding = embedding.cpu().numpy()
    # Calculate the scale of the noise (b = sensitivity / epsilon)
    scale = sensitivity / epsilon

    # Generate Laplacian noise with the same shape as the embedding
    noise = np.random.laplace(loc=0.0, scale=scale, size=embedding.shape)

    # Add noise to the original embedding
    private_embedding = embedding + noise

    if renormalize:
        # Renormalize the embedding to have unit length
        private_embedding /= np.linalg.norm(private_embedding)

    private_embedding = torch.tensor(private_embedding,dtype=torch.float32).to(device)

    return private_embedding


def inference(sample_sentence, model, projection, attacker_model, tokenizer, use_sampling=False, top_k=0, noise=None, multiplier=None, concat=False, max_new_tokens=150, temperature=1.0, diff=False):
    """
    Optimized inference supporting robust batch processing and early stopping.
    """
    model.eval()
    projection.eval()
    attacker_model.eval()

    # --- 1. Robust Input Handling ---
    # Ensure input is always a list
    if isinstance(sample_sentence, str):
        sample_sentence = [sample_sentence]
    elif not isinstance(sample_sentence, list):
        raise ValueError("Input must be a string or a list of strings.")

    # Apply 'concat' logic to each item in the batch
    if concat:
        processed = []
        for s in sample_sentence:
            if s.strip().endswith('.'):
                processed.append(s * 2)
            else:
                processed.append(s + ". " + s + ".")
        sample_sentence = processed

    # Detect device from the model
    device = model.device

    print(f"Processing batch size: {len(sample_sentence)}")

    with torch.no_grad():
        # --- 2. Encode & Project ---
        # model.encode typically handles lists. Ensure it returns a tensor on the correct device.
        embeddings = model.encode(sample_sentence, convert_to_tensor=True).to(device)

        if noise is not None:
            embeddings = embeddings - noise

        if multiplier:
            embeddings = embeddings * multiplier

        if diff:
            print("Applying Differential Privacy")
            embeddings = Differential(embeddings)

        # Project: [batch, hidden_dim] -> [batch, 1, hidden_dim]
        embeddings = projection(embeddings).unsqueeze(1)

        # --- 3. Generation Setup ---
        generated_embeds = embeddings
        past_key_values = None
        output_ids = None

        # Mask: 1 means "still generating", 0 means "finished"
        # We use this to stop the loop early and pad finished sequences
        unfinished_sequences = torch.ones(embeddings.shape[0], dtype=torch.long, device=device) # a list of length <batches>
        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

        # --- 4. Generation Loop ---
        for step in range(max_new_tokens):
            outputs = attacker_model(
                inputs_embeds=generated_embeds,
                past_key_values=past_key_values,
                use_cache=True,
                return_dict=True
            )

            logits = outputs.logits               # [batch, 1, vocab_size]
            past_key_values = outputs.past_key_values
            next_token_logits = logits[:, -1, :]  # [batch, vocab_size]

            # --- Sampling Logic ---
            if use_sampling:
                if temperature > 0:
                    next_token_logits = next_token_logits / temperature

                if top_k > 0:
                    top_logits, _ = torch.topk(next_token_logits, top_k)
                    min_logits = top_logits[:, -1].unsqueeze(-1)
                    next_token_logits = torch.where(
                        next_token_logits < min_logits,
                        torch.full_like(next_token_logits, -float('Inf')),
                        next_token_logits
                    )

                probs = torch.softmax(next_token_logits, dim=-1)
                next_token_id = torch.multinomial(probs, num_samples=1)
            else:
                next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

            # --- Batch Management ---

            # 1. Update finished mask
            # If token is EOS, mask becomes 0. If already 0, stays 0.
            is_eos = (next_token_id.squeeze(-1) == tokenizer.eos_token_id).long()  # a list of bool indicating if the generated token was eos
            unfinished_sequences = unfinished_sequences.mul(1 - is_eos) # element wise multiplication

            # 2. Append tokens to output
            if output_ids is None:
                output_ids = next_token_id
            else:
                output_ids = torch.cat((output_ids, next_token_id), dim=1)

            # 3. Global Stop: If all sequences are finished, break
            if unfinished_sequences.max() == 0:
                break

            # 4. Prepare input for next step
            # If a sequence is finished, feed it the PAD token.
            # This prevents the model from hallucinating based on previous garbage tokens.
            next_input_ids = next_token_id.clone()

            # Apply mask: Valid tokens keep their ID, finished ones become PAD
            # reshape unfinished_sequences to [batch, 1] for broadcasting
            mask_expanded = unfinished_sequences.unsqueeze(-1)
            next_input_ids = next_input_ids * mask_expanded + pad_token_id * (1 - mask_expanded)  # ensure that stopped sequences only received PAD tokens

            # Get embeddings for the next step
            next_token_emb = attacker_model.transformer.wte(next_input_ids)
            generated_embeds = next_token_emb

        # --- 5. Decode ---
        # batch_decode handles the list automatically
        result = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        return result

### Some testing

In [8]:
print(model)
"""After 10 epochs the model has overfit the training data, generalization to external data is affected"""
noise_sent = "HCMUT is a member of Vietnam National University, Ho Chi Minh City. It focuses on high technology and engineering."
# noised = model.encode("HCMUT is a member of Vietnam National University, Ho Chi Minh City. It focuses on high technology and engineering.", convert_to_tensor=True).to(device)
inference(["but i do take fridays off , when the weather gets warm . aka summer .", "wow that is great , i use to be in the military . ","cool ! i am more of a sports fan , love watching tennis or golf on tv"], model, projection, attacker_model, tokenizer, diff=True)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'T5EncoderModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Normalize()
)
Processing batch size: 3
Applying Differential Privacy


['i me!!! i mean i like to take naps in my job but what about movies?',
 ' do veryh do very veryh hhhh',
 ' the ; ; ; ; ; ; ; ; Authority Authority Authority Authority']

### Run desalting and multipler inference on the whole Personachat dataset
With various salt variant:
- Long salt
- Short salt
- Medium salt (maybe)

With various multiplers:
- 1.5
- 2
- 3

In [9]:
#@title Inference
salt_variants = {}
# salt_variants['long'] = 'HCMUT is a member of Vietnam National University, Ho Chi Minh City. It focuses on high technology and engineering.'
# salt_variants['short'] = 'HCMUT is a university.'
# salt_variants['medium'] = 'HCMUT is a university specializing in technology and engineering.'

# multipliers = [1.5, 2, 3]


# --- Configuration ---
batch_size = 50
save_interval = 1000  # Save every 1000 samples

# Ensure models are in eval mode globally to save overhead
model.eval()
projection.eval()
attacker_model.eval()

path = f'Inference-full'

for i in range(1):
    print(f"--- Processing the normal dataset ---")

    save_name = f'Personachat-Differential'
    collected_data = []

    # 1. Pre-calculate noise for this variant
    # No salt need for multiplier
    # with torch.no_grad()
    #     salt_text = salt_variants[variant]
    #     # Ensure noise is on device and has correct shape for broadcasting
    #     # model.encode usually returns [Dim] or [1, Dim].
    #     # We want to ensure it broadcasts over the batch later.
    #     noised = model.encode(salt_text, convert_to_tensor=True).to(device)

    # 2. Iterate in Batches
    total_samples = len(sent_list)
    i = 0

    # sent_list_mod = [sent.strip() + ' ' + salt_variants[variant] if sent.strip().endswith('.') else
    #                  sent + '. ' + salt_variants[variant] for sent in sent_list]

    sent_list_mod = sent_list

    try:
      for i in tqdm(range(0, total_samples, batch_size), desc="Processing batches"):
          # Slice the list to get current batch
          batch_sents = sent_list_mod[i : i + batch_size]

          # Run inference on the whole batch at once
          # Returns a LIST of strings
          batch_results = inference(
              sample_sentence=batch_sents,
              model=model,
              projection=projection,
              attacker_model=attacker_model,
              tokenizer=tokenizer,
              max_new_tokens=150,
              diff=True
          )

          # Pair inputs with outputs and store
          for og_sent, input_sent, output_text in zip(sent_list[i: i + batch_size], batch_sents, batch_results):
              collected_data.append({
                  'Original': og_sent,
                  'Input_Sentence': input_sent,
                  'Output': output_text,
                  # 'Multiplier': multiplier
              })

          # 3. Incremental Save
          # Check if we crossed a save_interval threshold
          current_count = i + len(batch_sents)
          if current_count % save_interval == 0:
              df = pd.DataFrame(collected_data)
              with pd.ExcelWriter(os.path.join(path, f"{save_name}-latest.xlsx"), engine="xlsxwriter") as writer: # i indicates where we should continue
                df.to_excel(writer, sheet_name="Base", index=False)
              print(f"  [Check] Processed {current_count}/{total_samples} samples...")
    except Exception as e:
      print(f"Error at index {i}: {e}")
      with pd.ExcelWriter(os.path.join(path, f"{save_name}-latest.xlsx"), engine="xlsxwriter") as writer: # i indicates where we should continue
        df = pd.DataFrame(collected_data)
        df.to_excel(writer, sheet_name="Base", index=False)
      print(f"  [Check] Processed {current_count}/{total_samples} samples...")


    # 4. Final Save for this variant
    with pd.ExcelWriter(os.path.join(path, f"{save_name}-full.xlsx"), engine="xlsxwriter") as writer: # i indicates where we should continue
      df = pd.DataFrame(collected_data)
      df.to_excel(writer, sheet_name="Base", index=False)
    print(f"  [Check] Processed {current_count}/{total_samples} samples...")
    print(f"Completed. Total saved: {len(df)}\n")




--- Processing the normal dataset ---


Processing batches:   0%|          | 0/2654 [00:00<?, ?it/s]

Processing batch size: 50
Applying Differential Privacy


Processing batches:   0%|          | 1/2654 [00:04<3:30:05,  4.75s/it]

Processing batch size: 50
Applying Differential Privacy


...

Processing batch size: 50
Applying Differential Privacy


Processing batches: 100%|█████████▉| 2653/2654 [3:41:44<00:05,  5.27s/it]

Processing batch size: 44
Applying Differential Privacy


Processing batches: 100%|██████████| 2654/2654 [3:41:49<00:00,  5.01s/it]


  [Check] Processed 132694/132694 samples...
Completed. Total saved: 132694



## Load and evaluate

In [6]:
salt_variants = {}
# salt_variants['long'] = 'HCMUT is a member of Vietnam National University, Ho Chi Minh City. It focuses on high technology and engineering.'
# salt_variants['short'] = 'HCMUT is a university.'
# salt_variants['medium'] = 'HCMUT is a university specializing in technology and engineering.'

# multipliers = [1.5, 2, 3]

def evaluate(df, sbert_model, bertscore_func, rouge_l_func, batch_size=512):
    sbert_scores = []
    bert_f1_scores = []
    rouge_l_scores = []

    # Convert DataFrame columns to lists
    originals = df['Original'].tolist()
    desalted_outputs = df['Output'].tolist()

    # --- 1. Calculate BERTScore F1 in Batches ---
    print(f"Calculating BERTScore F1 (Batch size: {batch_size})...")

    for i in tqdm(range(0, len(originals), batch_size), desc="BERTScore Batches"):
        # Slice the lists to create a chunk
        batch_originals = originals[i : i + batch_size]
        batch_outputs = desalted_outputs[i : i + batch_size]

        # Calculate score for this chunk
        # Assuming bertscore_func returns a list or tensor of scores
        batch_scores = bertscore_func(batch_outputs, batch_originals)

        # If the result is a tensor (common with huggingface), convert to list
        if hasattr(batch_scores, 'tolist'):
            batch_scores = batch_scores.tolist()

        bert_f1_scores.extend(batch_scores)

        # Optional: Clear CUDA cache if using GPU to strictly manage memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # --- 2. Calculate SBERT Cosine and ROUGE-L ---
    print("Calculating SBERT Cosine and ROUGE-L...")
    for i in tqdm(range(len(df)), desc="Evaluating sentences"):
        original_sent = originals[i]
        desalted_output = desalted_outputs[i]

        # SBERT Cosine Similarity
        # (Make sure sbert_cosine is defined or passed in your actual context)
        # Assuming sbert_cosine uses sbert_model internally or is a closure
        sbert_scores.append(sbert_cosine(original_sent, desalted_output))

        # ROUGE-L
        rouge_l_scores.append(rouge_l_func(original_sent, desalted_output))

    # Assign columns
    df['SBERT_Cosine_Similarity'] = sbert_scores
    df['BERTScore_F1'] = bert_f1_scores
    df['ROUGE_L'] = rouge_l_scores

    return df



In [7]:
# Load the DataFrame from the saved Excel file (if not already loaded)
for i in range(1):
  print(f"Evaluating Normal")
  path = f'Inference-full/ALGEN/Desalting_output.tsv'
  df_evaluated = pd.read_csv(path, sep='\t') # Added sep='\t' for tab-separated values
  df_evaluated['Output'] = df_evaluated['Desalted_Output'].astype(str)

  # Evaluate the results
  df_evaluated = evaluate(df_evaluated, sbert, bertscore_f1, rouge_l)

  # Define the path to save the evaluated DataFrame
  save_path_evaluated = f'Inference-full/ALGEN/Desalting-evaluated.xlsx'

  # Save the DataFrame with evaluation scores
  with pd.ExcelWriter(save_path_evaluated, engine='xlsxwriter') as writer:
      df_evaluated.to_excel(writer, sheet_name='Evaluated_Results', index=False)

  print(f"Evaluation complete. Results saved to {save_path_evaluated}")
  display(df_evaluated.head())

Evaluating Normal
Calculating BERTScore F1 (Batch size: 512)...


BERTScore Batches:   0%|          | 0/118 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

BERTScore Batches: 100%|██████████| 118/118 [17:40<00:00,  8.99s/it]


Calculating SBERT Cosine and ROUGE-L...


Evaluating sentences: 100%|██████████| 60000/60000 [12:35<00:00, 79.39it/s]


Evaluation complete. Results saved to Inference-full/ALGEN/Desalting-evaluated.xlsx


,Original,Input_Sentence,Desalted_Output,Variant,Output,SBERT_Cosine_Similarity,BERTScore_F1,ROUGE_L
0,"hi , how are you doing ? i am getting ready to...","hi , how are you doing ? i am getting ready to...",I am a I I I I I I do I I need I want I are,long,I am a I I I I I I do I I need I want I are,0.141599,-0.406055,0.171429
1,you must be very fast . hunting is one of my f...,you must be very fast . hunting is one of my f...,have a i want a new i love a love i i need a do i,long,have a i want a new i love a love i i need a do i,0.069763,-0.102531,0.000000
2,i am ! for my hobby i like to do canning or so...,i am ! for my hobby i like to do canning or so...,a i a need need love city am need love,long,a i a need need love city am need love,0.022895,-0.304308,0.173913
3,i also remodel homes when i am not out bow hun...,i also remodel homes when i am not out bow hun...,need a new apartment i am i need i have a hous...,long,need a new apartment i am i need i have a hous...,0.350363,-0.125245,0.148148
4,that is neat . when i was in high school i pla...,that is neat . when i was in high school i pla...,a new apartment a real a a housing a good a ho...,long,a new apartment a real a a housing a good a ho...,0.074273,-0.180712,0.000000


In [11]:
path = f'Inference-full/ALGEN/Scaling_output.tsv'
df = pd.read_csv(path, sep='\t')

In [17]:
for i in range(1):
  print(f"Evaluating Normal")
  path = f'Inference-full/ALGEN/Scaling_output.tsv'
  df_evaluated = pd.read_csv(path, sep='\t') # Added sep='\t' for tab-separated values
  df_evaluated['Output'] = df_evaluated['Scaled_Output'].astype(str)

  # Evaluate the results
  df_evaluated = evaluate(df_evaluated, sbert, bertscore_f1, rouge_l)

  # Define the path to save the evaluated DataFrame
  save_path_evaluated = f'Inference-full/ALGEN/Scaling-evaluated.xlsx'

  # Save the DataFrame with evaluation scores
  with pd.ExcelWriter(save_path_evaluated, engine='xlsxwriter') as writer:
      df_evaluated.to_excel(writer, sheet_name='Evaluated_Results', index=False)

  print(f"Evaluation complete. Results saved to {save_path_evaluated}")
  display(df_evaluated.head())

Evaluating Normal
Calculating BERTScore F1 (Batch size: 512)...


BERTScore Batches: 100%|██████████| 196/196 [24:56<00:00,  7.64s/it]


Calculating SBERT Cosine and ROUGE-L...


Evaluating sentences: 100%|██████████| 100000/100000 [21:20<00:00, 78.07it/s]


Evaluation complete. Results saved to Inference-full/ALGEN/Scaling-evaluated.xlsx


,Original,Input_Sentence,Scaled_Output,Scale_Factor,Output,SBERT_Cosine_Similarity,BERTScore_F1,ROUGE_L
0,"hi , how are you doing ? i am getting ready to...","hi , how are you doing ? i am getting ready to...",need a a good a need i want a they need re i n...,0.5,need a a good a need i want a they need re i n...,0.156706,-0.295519,0.058824
1,you must be very fast . hunting is one of my f...,you must be very fast . hunting is one of my f...,want a a love love a want i love i need a have...,0.5,want a a love love a want i love i need a have...,0.070797,-0.401273,0.000000
2,i am ! for my hobby i like to do canning or so...,i am ! for my hobby i like to do canning or so...,need a i want a need i i would like a do i hav...,0.5,need a i want a need i i would like a do i hav...,0.164396,-0.349462,0.275862
3,i also remodel homes when i am not out bow hun...,i also remodel homes when i am not out bow hun...,a new apartment i a apartments i new a i love ...,0.5,a new apartment i a apartments i new a i love ...,0.249086,-0.112920,0.153846
4,that is neat . when i was in high school i pla...,that is neat . when i was in high school i pla...,a new apartment a a real a housing a good a ho...,0.5,a new apartment a a real a housing a good a ho...,0.090313,-0.197388,0.000000


### Test inference DialoGPT in Vietnamese

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def chat_with_dialogpt():
    # 1. Load Tokenizer and Model
    # You can change 'medium' to 'small' or 'large' depending on your hardware
    model_name = "microsoft/DialoGPT-large"
    print(f"Loading {model_name}...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    print("Model loaded! Type 'quit' to exit.")
    print("-----------------------------------")

    # Initialize chat history to None
    chat_history_ids = None
    attention_mask_history = None

    # 2. Chat Loop
    step = 0
    while True:
        # Get user input
        user_input = input(">> User: ")

        if user_input.lower() in ["quit", "exit"]:
            break

        # encode the new user input, add the eos_token and return a tensor in Pytorch
        encoded_input = tokenizer(user_input + tokenizer.eos_token, return_tensors='pt')
        new_user_input_ids = encoded_input['input_ids']
        new_attention_mask = encoded_input['attention_mask']

        # append the new user input tokens to the chat history
        if step > 0:
            bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
            attention_mask_ids = torch.cat([attention_mask_history, new_attention_mask], dim=-1)
        else:
            bot_input_ids = new_user_input_ids
            attention_mask_ids = new_attention_mask

        # 3. Generate a response
        # We use strict parameters (top_k, top_p) to make the text sound more human-like and less repetitive
        chat_history_ids = model.generate(
            bot_input_ids,
            max_length=1000,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,
            do_sample=True,
            top_k=100,
            top_p=0.7,
            temperature=0.8,
            attention_mask=attention_mask_ids # Pass the attention mask
        )

        # Update the attention mask history for the next turn
        attention_mask_history = torch.ones_like(chat_history_ids)

        # 4. Decode and print response
        # We need to slice the output to get only the new tokens generated by the bot
        # The model returns [History + User Input + Bot Response], so we slice off the input part
        response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

        print(f"DialoGPT: {response}")

        step += 1

if __name__ == "__main__":
    chat_with_dialogpt()


Loading microsoft/DialoGPT-large...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loaded! Type 'quit' to exit.
-----------------------------------
>> User: được một thứ gì đó người ta không muốn thấy
DialoGPT: This is the correct spelling.
>> User: what language did i just speak?
DialoGPT: You just spoke a sentence that you wrote in English.
>> User: Xin chào
DialoGPT: I can't even remember what that is supposed to say.
>> User: đây là tiếng Việt
DialoGPT: You can't just read the name of the word.
>> User: what name?
DialoGPT: What is this?


KeyboardInterrupt: Interrupted by user